In [ ]:
import requests 
import pandas as pd
import time
import datetime
from openpyxl import Workbook


url = "https://api.coingecko.com/api/v3/coins/markets"
param = {
    "vs_currency": "usd",
    "order": "market_cap_desc",
    "per_page": 50,
    "page": 1,
    "sparkline": False
}

while True:
    
    response = requests.get(url, params=param)
    
    if response.status_code == 200:
        print("Connection Successfully! \nGetting Data....")
    
        # Storing the response in the data
        data = response.json()
    
        # Creating data frame
        df = pd.DataFrame(data)
    
        #convert data frame 
        df = df[["name", "symbol", "current_price", "market_cap", "total_volume", "price_change_percentage_24h"]]
    
        # Identify Top 5 Cryptos by Market Cap
        top_5 = df.nlargest(5, 'market_cap')[["name", "market_cap"]]

        # Calculate Average Price of Top 50 Cryptos
        avg_price = df["current_price"].mean()
    
        # Find Highest & Lowest 24h % Price Change
        highest_change = df.loc[df["price_change_percentage_24h"].idxmax(), ["name", "price_change_percentage_24h"]]
        lowest_change = df.loc[df["price_change_percentage_24h"].idxmin(), ["name", "price_change_percentage_24h"]]

        # create file name
        filename = f"crypto_live_data_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    
        # Save data in excel.
        with pd.ExcelWriter(filename, engine="openpyxl") as writer:
            df.to_excel(writer, sheet_name="Top 50 Cryptos", index=False)
            top_5.to_excel(writer, sheet_name="Top 5 by Market Cap", index=False)
    
            summary = pd.DataFrame({
                "Metric": ["Average Price", "Highest 24h Change", "Lowest 24h Change"],
                "Value": [
                    avg_price, 
                    f"{highest_change['name']} ({highest_change['price_change_percentage_24h']}%)", 
                    f"{lowest_change['name']} ({lowest_change['price_change_percentage_24h']}%)"
                ]
            })
            summary.to_excel(writer, sheet_name="Analysis Summary", index=False)
    
        print(f"Excel saved as {filename}")
        
        time.sleep(300)  # Wait for 5 minutes before next update
    
    else:
        print(f"Connecction failed error code {response.status_code}")


Connection Successfully! 
Getting Data....
Excel saved as crypto_live_data_20250213_164200.xlsx
Connection Successfully! 
Getting Data....
Excel saved as crypto_live_data_20250213_164701.xlsx


In [ ]:


    #print("Top 50 data \n",df)
    #print("\n",top_5,"\n",avg_price,"\n", highest_change,"\n", lowest_change)

    """
    # Saving the data
    df.to_csv(f"Crypto data.csv",index=False)
    print("Data Saved Successfully..")"""